# Проект RFM-анализ

In [116]:
import pandas as pd
import numpy as np

**Подготовка данных**

In [3]:
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
stud_assessment = pd.read_csv('studentAssessment.csv')
stud_reg = pd.read_csv('studentRegistration.csv')

In [4]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [5]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [6]:
stud_assessment[stud_assessment['date_submitted'] < 0]

,id_assessment,id_student,date_submitted,is_banked,score
1641,1758,603861,-1,1,61.0
1685,1758,1970390,-1,1,78.0
1690,1758,2073016,-1,1,60.0
1693,1758,2139353,-1,1,65.0
1708,1758,2358969,-1,1,68.0
...,...,...,...,...,...
172516,37440,624767,-1,1,100.0
172526,37440,631446,-1,1,100.0
172717,37441,624767,-1,1,100.0
172727,37441,631446,-1,1,100.0


In [7]:
stud_reg.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


**Кейс 1**\
Сколько студентов успешно сдали только один курс?

In [11]:
stud_assessment.query('score >= 40')\
                .merge(assessments, how = 'inner', on = 'id_assessment') \
                .query('assessment_type == "Exam"').groupby('id_student') \
                .agg(num_of_exams=('id_assessment', 'nunique')) \
                .query('num_of_exams == 1').count()

num_of_exams    3802
dtype: int64

**Кейс 2**\
Найдите и отсортируйте id экзаменов в рамках курса по возрастанию уровня завершаемости: \
_завершаемость = кол-во успешных экзаменов/кол-во всех попыток сдать экзамен * 100_

In [15]:
stud_asses_exams = stud_assessment.merge(assessments, how = 'inner', on = 'id_assessment').query('assessment_type == "Exam"')

In [16]:
stud_asses_exams.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
52923,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0


In [88]:
(stud_asses_exams.query('score >= 40')
                .groupby('id_assessment')
                .id_student.nunique() / 
 stud_asses_exams.query('score < 40')
                .groupby('id_assessment')
                .id_student.nunique()).sort_values(0)

id_assessment
25340     5.142857
24299     6.838926
25368     7.796296
24290     8.000000
25354     9.755556
25361    12.435897
Name: id_student, dtype: float64

In [108]:
(stud_assessment
            .merge(
                    assessments.query('assessment_type == "Exam"')['id_assessment', 'code_module'],
                    how = 'inner',
                    on = 'id_assessment')
            .assign(success = lambda x: (x['score'] >= 40))
            .groupby('id_assessment', as_index = False) 
            .agg(all_attempts = ('success', 'count'), success_count = ('success', 'sum'))
            .assign(completion_rate = lambda x: (x['success_count'] / x['all_attempts']))
            .sort_values(by='completion_rate')
)

,id_assessment,all_attempts,success_count,completion_rate
2,25340,602,504,0.837209
1,24299,1168,1019,0.872432
5,25368,950,842,0.886316
0,24290,747,664,0.888889
3,25354,968,878,0.907025
4,25361,524,485,0.925573


**Кейс 3**\
Выявите самые популярные предметы (ТОП-3) по количеству регистраций на них:

In [96]:
(stud_reg
    .groupby('code_module', as_index = False)
    .agg({'date_registration' : 'count'})
    .sort_values(by='date_registration', ascending = False)
    .head(3))

,code_module,date_registration
1,BBB,7900
5,FFF,7751
3,DDD,6257


**Кейс 4**\
Выявите предметы с самым большим оттоком* (ТОП-3): \
_Отток — это отмена регистрации на предмет._

In [106]:
(stud_reg
    .query('date_unregistration.notnull()')
    .groupby('code_module', as_index = False)
    .agg({'date_unregistration' : 'count'})
    .sort_values(by='date_unregistration', ascending = False)
    .head(3))

,code_module,date_unregistration
5,FFF,2380
1,BBB,2377
3,DDD,2235


**Кейс 5**\
В период с начала 2013 по конец 2014 выявите семестр с самой низкой завершаемостью курсов:

In [122]:
(stud_assessment
            .merge(
                    assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_module']],
                    how = 'inner',
                    on = 'id_assessment')
            .merge (courses[['code_module', 'code_presentation']],
                    how = 'inner',
                    on = 'code_module')
            .assign(success = lambda x: (x['score'] >= 40))
            .groupby('code_presentation', as_index = False) 
            .agg(
            {
                'success': [np.mean]
            })
)

,code_presentation,success
,,mean
0,2013B,0.889947
1,2013J,0.889947
2,2014B,0.885662
3,2014J,0.885662


**Кейс 7**\
В период с начала 2013 по конец 2014 выявите семестр с самыми долгими средними сроками сдачи курсов:

In [142]:
(stud_assessment
            .merge(
                    assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']],
                    how = 'inner',
                    on = 'id_assessment')
            .assign(success = lambda x: (x['score'] >= 40))
            .groupby('code_presentation', as_index = False) 
            .agg(success_date_submitted = ('date_submitted', 'mean'))
            .sort_values(by ='success_date_submitted', ascending = False)
)

,code_presentation,success_date_submitted
3,2014J,243.674221
1,2013J,239.505165
2,2014B,232.988985
0,2013B,230.179402


**Кейс 7**\
Постройте адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можете выбрать следующие метрики:

R — среднее время сдачи одного экзамена,

F — завершаемость курсов,

M — среднее количество баллов, получаемое за экзамен.

Для каждого RFM-сегмента постройте границы метрик recency, frequency и monetary для интерпретации этих кластеров.

In [220]:
students_rfm = (stud_assessment
            .merge(
                    assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']],
                    how = 'inner',
                    on = 'id_assessment')
            .assign(success = lambda x: (x.score >= 40))
            .groupby('id_student')
            .agg(recency = ('date_submitted', 'mean'), frequency = ('success', 'mean'), monetary = ('score', 'mean'))
)

In [221]:
students_rfm.frequency = students_rfm.frequency.mul(100)

**Кейс 8**\
Используя логику:

1) по recency можно получить 2, если значение recency меньше либо равно медиане по recency. В остальных случаях — 1.

2) по frequency можно получить 1, если значение frequency меньше 50. Можно получить 2, если значение по frequency меньше 100. Можно получить 3 в остальных случаях.

3) по monetary можно получить 1, если значение monetary меньше 40. Можно получить 2, если значение по monetary меньше либо равно 80. Можно получить 3 в остальных случаях.

Сколько клиентов попадут в кластер 232?

In [210]:
students_rfm.query('frequency < 50')

,recency,frequency,monetary,F
id_student,,,,
31173,242.0,0.0,27.0,Unknown
46753,234.0,0.0,36.0,Unknown
51955,242.0,0.0,22.0,Unknown
52014,231.0,0.0,13.0,Unknown
63566,237.0,0.0,36.0,Unknown
...,...,...,...,...
2611779,230.0,0.0,24.0,Unknown
2615956,243.0,0.0,27.0,Unknown
2655683,230.0,0.0,33.0,Unknown


In [236]:
students_rfm['R'] = pd.qcut(students_rfm.recency, 2, labels = [2, 1])

students_rfm.loc[students_rfm['frequency'].between(students_rfm.frequency.min(), 50, inclusive='left'), 'F'] = 1
students_rfm.loc[students_rfm['frequency'].between(40, 80, inclusive='left'), 'F'] = 2
students_rfm.loc[students_rfm['frequency'].between(100, students_rfm.frequency.max(), inclusive='both'), 'F'] = 3
students_rfm['M'] = 'Unknown'
students_rfm.loc[students_rfm['monetary'].between(students_rfm.monetary.min(), 40, inclusive='left'), 'M'] = 1
students_rfm.loc[students_rfm['monetary'].between(40, 80, inclusive='both'), 'M'] = 2
students_rfm.loc[students_rfm['monetary'].between(80.5, students_rfm.monetary.max(), inclusive= 'both'), 'M'] = 3

In [238]:
students_rfm.query("R == 2 & ")

,recency,frequency,monetary,R,F,M
id_student,,,,,,
23698,243.0,100.0,80.0,1,3.0,2
24213,236.0,100.0,58.0,2,3.0,2
27116,243.0,100.0,96.0,1,3.0,3
28046,237.0,100.0,40.0,2,3.0,2
28787,243.0,100.0,44.0,1,3.0,2
...,...,...,...,...,...,...
2694886,236.0,100.0,69.0,2,3.0,2
2694933,230.0,100.0,73.0,2,3.0,2
2695608,237.0,100.0,73.0,2,3.0,2


In [219]:
students_rfm.recency.median()

241.0